# User Funnel & Retention Analysis (Product Analytics, SQL)

This notebook generates synthetic event logs and computes:
- Funnel conversion (signup → onboarding → activation → engagement)
- Cohort retention (D1, D7)
- Segment cuts by country and device

SQL versions are provided in `sql/`.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.generate_events import generate_events

events = generate_events(n_users=60000, days=21, seed=42)
events.head(), events.shape


## 1) Funnel Analysis


In [ ]:
first = (events
         .pivot_table(index='user_id', columns='event_name', values='event_time', aggfunc='min')
         .reset_index())

first['signed_up'] = first['signup'].notna().astype(int)
first['onboarded'] = first['onboarding_complete'].notna().astype(int)
first['activated'] = first[['add_friend','follow_creator']].notna().any(axis=1).astype(int)
first['engaged'] = first[['view_feed','like','comment','share']].notna().any(axis=1).astype(int)

summary = pd.DataFrame({
    'step': ['signed_up','onboarded','activated','engaged'],
    'users': [first['signed_up'].sum(), first['onboarded'].sum(), first['activated'].sum(), first['engaged'].sum()],
})
summary['conversion_from_signup'] = summary['users'] / summary.loc[0,'users']
summary


In [ ]:
ax = summary.set_index('step')['conversion_from_signup'].plot(kind='bar')
plt.title('Funnel Conversion (from Signup)')
plt.ylabel('Rate')
plt.xticks(rotation=0)
plt.ylim(0, 1)
plt.show()


## 2) Cohort Retention (D1 / D7)


In [ ]:
signup = (events[events.event_name=='signup']
          .groupby('user_id', as_index=False)['event_date'].min()
          .rename(columns={'event_date':'signup_date'}))

activity = events[events.event_name.isin(['view_feed','like','comment','share'])][['user_id','event_date']].drop_duplicates()

ret = signup.merge(activity, on='user_id', how='left')
ret['event_date'] = pd.to_datetime(ret['event_date'])
ret['signup_date'] = pd.to_datetime(ret['signup_date'])

ret['d1'] = (ret['event_date'] == (ret['signup_date'] + pd.Timedelta(days=1))).astype(int)
ret['d7'] = (ret['event_date'] == (ret['signup_date'] + pd.Timedelta(days=7))).astype(int)

user_ret = ret.groupby(['user_id','signup_date'], as_index=False).agg(d1=('d1','max'), d7=('d7','max'))
cohorts = user_ret.groupby('signup_date', as_index=False).agg(
    cohort_users=('user_id','count'),
    d1_retention=('d1','mean'),
    d7_retention=('d7','mean')
)
cohorts.head(10)


In [ ]:
plt.figure()
plt.plot(cohorts['signup_date'], cohorts['d1_retention'], label='D1')
plt.plot(cohorts['signup_date'], cohorts['d7_retention'], label='D7')
plt.title('Cohort Retention Over Signup Date')
plt.ylabel('Retention')
plt.xlabel('Signup Date')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## 3) Segment Breakdown (Country, Device)


In [ ]:
user_dim = (events[events.event_name=='signup']
            .sort_values('event_time')
            .drop_duplicates('user_id')[['user_id','country','device_type']])

seg = first[['user_id','onboarded','activated','engaged']].merge(user_dim, on='user_id', how='left')
seg_summary = seg.groupby(['country','device_type'], as_index=False).agg(
    users=('user_id','count'),
    onboarding_rate=('onboarded','mean'),
    activation_rate=('activated','mean'),
    engagement_rate=('engaged','mean')
).sort_values('users', ascending=False)
seg_summary.head(15)


## 4) Recommendations (Example)
- Improve onboarding steps where the funnel drops the most.
- Test activation nudges (e.g., follow suggestions) for users who complete onboarding.
- Monitor cohort retention and segment results by device/country to find uneven experiences.
